![tower bridge](london.jpg)

London, or as the Romans called it "Londonium"! Home to [over 8.5 million residents](https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/bulletins/populationandhouseholdestimatesenglandandwales/census2021unroundeddata#population-and-household-estimates-england-and-wales-data) who speak over [300 languages](https://web.archive.org/web/20080924084621/http://www.cilt.org.uk/faqs/langspoken.htm). While the City of London is a little over one square mile (hence its nickname "The Square Mile"), Greater London has grown to encompass 32 boroughs spanning a total area of 606 square miles! 

![underground train leaving a platform](tube.jpg)

Given the city's roads were originally designed for horse and cart, this area and population growth has required the development of an efficient public transport system! Since the year 2000, this has been through the local government body called **Transport for London**, or *TfL*, which is managed by the London Mayor's office. Their remit covers the London Underground, Overground, Docklands Light Railway (DLR), buses, trams, river services (clipper and [Emirates Airline cable car](https://en.wikipedia.org/wiki/London_cable_car)), roads, and even taxis.

The Mayor of London's office make their data available to the public [here](https://data.london.gov.uk/dataset). In this project, you will work with a slightly modified version of a dataset containing information about public transport journey volume by transport type. 

The data has been loaded into a **Snowflake** database called `TFL` with a single table called `JOURNEYS`, including the following data:

## TFL.JOURNEYS

| Column | Definition | Data type |
|--------|------------|-----------|
| `MONTH`| Month in number format, e.g., `1` equals January | `INTEGER` |
| `YEAR` | Year | `INTEGER` |
| `DAYS` | Number of days in the given month | `INTEGER` |
| `REPORT_DATE` | Date that the data was reported | `DATE` |
| `JOURNEY_TYPE` | Method of transport used | `VARCHAR` |
| `JOURNEYS_MILLIONS` | Millions of journeys, measured in decimals | `FLOAT` |

Note that *in Snowflake all databases, tables, and columns are **upper case*** by default.

You will execute SQL queries to answer three questions, as listed in the instructions.

In [49]:
WITH no_nulls AS (
SELECT 	
	*	
FROM 
	TFL.JOURNEYS
WHERE 
	TFL.JOURNEYS.MONTH IS NOT NULL AND
	TFL.JOURNEYS.YEAR IS NOT NULL AND
	TFL.JOURNEYS.DAYS IS NOT NULL AND
	TFL.JOURNEYS.REPORT_DATE IS NOT NULL AND
	TFL.JOURNEYS.JOURNEY_TYPE IS NOT NULL AND
	TFL.JOURNEYS.JOURNEYS_MILLIONS IS NOT NULL
)

SELECT 
	JOURNEY_TYPE,
	ROUND(SUM(JOURNEYS_MILLIONS),2) total_journeys_millions
FROM 
	NO_NULLS 
GROUP BY  
	JOURNEY_TYPE
ORDER BY 
	total_journeys_millions DESC



,JOURNEY_TYPE,TOTAL_JOURNEYS_MILLIONS
0,Bus,24905.19
1,Underground & DLR,15020.47
2,Overground,1666.85
3,TfL Rail,411.31
4,Tram,314.69
5,Emirates Airline,14.58


In [50]:
WITH no_nulls AS (
	SELECT 
--Only Selecting the row I need for this CTE query saved about a second in query time. Hence not selecting * columns.
		TFL.JOURNEYS.MONTH,
		TFL.JOURNEYS.YEAR,
		TFL.JOURNEYS.JOURNEYS_MILLIONS
	FROM 
		TFL.JOURNEYS
	WHERE 
		TFL.JOURNEYS.MONTH IS NOT NULL AND
		TFL.JOURNEYS.YEAR IS NOT NULL AND
		TFL.JOURNEYS.DAYS IS NOT NULL AND
		TFL.JOURNEYS.REPORT_DATE IS NOT NULL AND
		TFL.JOURNEYS.JOURNEY_TYPE IS NOT NULL AND
		TFL.JOURNEYS.JOURNEYS_MILLIONS IS NOT NULL
)
SELECT 
	MONTH, 
	YEAR,
	ROUND(SUM(JOURNEYS_MILLIONS),2) most_popular_transport_types
FROM 
	no_nulls
GROUP BY  
	YEAR,
	MONTH
ORDER BY 
	YEAR DESC
--IF viewd by chart will be the most beneficial

,MONTH,YEAR,MOST_POPULAR_TRANSPORT_TYPES
0,11,2022,262.31
1,2,2022,248.52
2,9,2022,279.54
3,6,2022,242.65
4,1,2022,239.35
...,...,...,...
151,7,2010,287.06
152,5,2010,251.09
153,4,2010,278.11
154,3,2010,268.05


In [51]:
WITH no_nulls AS (
	SELECT 
		TFL.JOURNEYS.MONTH,
		TFL.JOURNEYS.YEAR,
		TFL.JOURNEYS.JOURNEYS_MILLIONS
	FROM 
		TFL.JOURNEYS
	WHERE 
		TFL.JOURNEYS.MONTH IS NOT NULL AND
		TFL.JOURNEYS.YEAR IS NOT NULL AND
		TFL.JOURNEYS.DAYS IS NOT NULL AND
		TFL.JOURNEYS.REPORT_DATE IS NOT NULL AND
		TFL.JOURNEYS.JOURNEY_TYPE IS NOT NULL AND JOURNEY_TYPE = 'Emirates Airline' AND
		TFL.JOURNEYS.JOURNEYS_MILLIONS IS NOT NULL
)
SELECT 
	MONTH, 
	YEAR,
	ROUND(SUM(JOURNEYS_MILLIONS),2) rounded_journeys_millions
FROM 
	no_nulls
GROUP BY  
	MONTH, 
	YEAR
ORDER BY 
	rounded_journeys_millions DESC
LIMIT 5



,MONTH,YEAR,ROUNDED_JOURNEYS_MILLIONS
0,5,2012,0.53
1,6,2012,0.38
2,4,2012,0.24
3,5,2013,0.19
4,5,2015,0.19


In [52]:
WITH top_months AS (
    SELECT 
        'Month' AS period,
        TFL.JOURNEYS.MONTH AS time_period,
        ROUND(SUM(TFL.JOURNEYS.JOURNEYS_MILLIONS), 2) AS most_popular_transport_types
    FROM 
        TFL.JOURNEYS 
    GROUP BY  
        TFL.JOURNEYS.MONTH
    ORDER BY 
        most_popular_transport_types DESC
    LIMIT 5
),
top_years AS (
    SELECT 
        'Year' AS period,
        TFL.JOURNEYS.YEAR AS time_period,
        ROUND(SUM(TFL.JOURNEYS.JOURNEYS_MILLIONS), 2) AS most_popular_transport_types
    FROM 
        TFL.JOURNEYS 
    GROUP BY  
        TFL.JOURNEYS.YEAR
    ORDER BY 
        most_popular_transport_types DESC
    LIMIT 5
)
SELECT * FROM top_months
UNION ALL
SELECT * FROM top_years;


,PERIOD,TIME_PERIOD,MOST_POPULAR_TRANSPORT_TYPES
0,Month,9,3847.42
1,Month,7,3840.90
2,Month,8,3750.63
3,Month,4,3627.31
4,Month,12,3587.20
5,Year,2015,3752.51
6,Year,2016,3714.69
7,Year,2017,3678.44
8,Year,2018,3672.71
9,Year,2019,3633.53


In [53]:
-- least_popular_years_tube
SELECT 
	TFL.JOURNEYS.YEAR,
	TFL.JOURNEYS.JOURNEY_TYPE,
	ROUND(SUM(TFL.JOURNEYS.JOURNEYS_MILLIONS),2) AS total_journeys_millions
FROM
	TFL.JOURNEYS
WHERE TFL.JOURNEYS.JOURNEY_TYPE ILIKE '%Underground%'
GROUP BY 
	TFL.JOURNEYS.YEAR,
	TFL.JOURNEYS.JOURNEY_TYPE
ORDER BY
		total_journeys_millions
LIMIT 5

,YEAR,JOURNEY_TYPE,TOTAL_JOURNEYS_MILLIONS
0,2020,Underground & DLR,310.18
1,2021,Underground & DLR,748.45
2,2022,Underground & DLR,1064.86
3,2010,Underground & DLR,1096.15
4,2011,Underground & DLR,1156.65
